In [17]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [18]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [19]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [20]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
"""
    Parameters
    ----------
    problem_formulation_id : int {0, ..., 5}
                             problem formulations differ with respect to the objectives
                             0: Total cost, and casualties
                             1: Expected damages, costs, and casualties
                             2: expected damages, dike investment costs, rfr costs, evacuation cost, and casualties
                             3: costs and casualties disaggregated over dike rings, and room for the river and evacuation costs
                             4: Expected damages, dike investment cost and casualties disaggregated over dike rings and room for the river and evacuation costs
                             5: disaggregate over time and space

    Notes
    -----
    problem formulations 4 and 5 rely on ArrayOutcomes and thus cannot straightforwardly
    be used in optimizations

    """
problem_formulation = 3
dike_model, planning_steps = get_model_for_problem_formulation(problem_formulation)

In [21]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [22]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [23]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x00000231286568C0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x00000231286568C0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x00000231286568C0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x00000231286568C0>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x00000231286568C0>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x00000231286568C0>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [24]:
# running the model through EMA workbench
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=2, policies=4)

[MainProcess/INFO] performing 2 scenarios * 4 policies * 1 model(s) = 8 experiments
100%|████████████████████████████████████████████| 8/8 [00:01<00:00,  4.63it/s]
[MainProcess/INFO] experiments finished


In [25]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,111,294.311276,10.0,0.370731,48.947046,1.0,0.041225,101.745798,10.0,0.640001,261.017542,1.5,0.389091,54.699063,10.0,0.521063,4.5,4.5,1.5,1,1,1,1,1,1,0,0,1,1,0,1,1,1,1,4,7,8,1,3,7,4,5,1,10,10,6,6,0,6,1,30,26,dikesnet
1,16,140.570342,1.0,0.911997,285.201774,10.0,0.818702,259.553224,1.5,0.223216,162.298730,10.0,0.834791,240.200410,1.0,0.046650,2.5,1.5,4.5,1,1,1,1,1,1,0,0,1,1,0,1,1,1,1,4,7,8,1,3,7,4,5,1,10,10,6,6,0,6,1,31,26,dikesnet
2,111,294.311276,10.0,0.370731,48.947046,1.0,0.041225,101.745798,10.0,0.640001,261.017542,1.5,0.389091,54.699063,10.0,0.521063,4.5,4.5,1.5,0,1,1,0,1,0,1,0,1,0,0,1,1,0,0,2,0,1,4,10,8,5,0,5,0,2,10,3,8,3,4,30,27,dikesnet
3,16,140.570342,1.0,0.911997,285.201774,10.0,0.818702,259.553224,1.5,0.223216,162.298730,10.0,0.834791,240.200410,1.0,0.046650,2.5,1.5,4.5,0,1,1,0,1,0,1,0,1,0,0,1,1,0,0,2,0,1,4,10,8,5,0,5,0,2,10,3,8,3,4,31,27,dikesnet
4,111,294.311276,10.0,0.370731,48.947046,1.0,0.041225,101.745798,10.0,0.640001,261.017542,1.5,0.389091,54.699063,10.0,0.521063,4.5,4.5,1.5,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,8,4,4,5,7,4,10,4,9,7,5,3,0,4,8,3,30,28,dikesnet
5,16,140.570342,1.0,0.911997,285.201774,10.0,0.818702,259.553224,1.5,0.223216,162.298730,10.0,0.834791,240.200410,1.0,0.046650,2.5,1.5,4.5,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,8,4,4,5,7,4,10,4,9,7,5,3,0,4,8,3,31,28,dikesnet
6,111,294.311276,10.0,0.370731,48.947046,1.0,0.041225,101.745798,10.0,0.640001,261.017542,1.5,0.389091,54.699063,10.0,0.521063,4.5,4.5,1.5,0,0,0,0,0,0,1,1,0,1,1,0,0,1,0,5,10,9,10,2,1,2,9,5,2,7,1,8,5,2,1,30,29,dikesnet
7,16,140.570342,1.0,0.911997,285.201774,10.0,0.818702,259.553224,1.5,0.223216,162.298730,10.0,0.834791,240.200410,1.0,0.046650,2.5,1.5,4.5,0,0,0,0,0,0,1,1,0,1,1,0,0,1,0,5,10,9,10,2,1,2,9,5,2,7,1,8,5,2,1,31,29,dikesnet


In [26]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,2.362054e+08,0.0,3.048226e+08,0.061076,8.681145e+07,0.000000,7.355511e+07,0.000000,8.954281e+07,0.000000,1.948600e+09,5485.709429
1,2.362054e+08,0.0,1.737042e+08,0.000732,8.681145e+07,0.000000,7.355511e+07,0.000000,1.005180e+08,0.003062,1.948600e+09,333.720938
2,7.222271e+07,0.0,3.093760e+08,0.001984,6.286508e+07,0.000000,3.129331e+07,0.000185,1.278784e+08,0.000000,8.257000e+08,1427.222305
3,7.222271e+07,0.0,2.972961e+08,0.000000,6.286508e+07,0.000000,2.978919e+07,0.000044,2.084277e+08,0.008432,8.257000e+08,5931.409015
4,2.105126e+08,0.0,2.336733e+08,0.001295,1.461625e+08,0.000000,4.091686e+07,0.000000,9.488414e+07,0.000983,9.282000e+08,1196.806629
5,2.105126e+08,0.0,2.259944e+08,0.000000,1.461625e+08,0.000000,4.091686e+07,0.000000,3.861582e+08,0.028719,9.282000e+08,18405.122860
6,3.076191e+08,0.0,2.166808e+08,0.000000,1.072753e+08,0.000000,3.113774e+07,0.000000,1.391353e+08,0.000000,5.599000e+08,0.000000
7,3.076191e+08,0.0,2.166808e+08,0.000000,1.111651e+08,0.002441,3.113774e+07,0.000000,1.391353e+08,0.000000,5.599000e+08,81.622430


In [27]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations
# policy 4 doe niks
'''
A1 = Doesburg Upstream
A2 = Cortenoever Upmidstream
A3 = Zutphen Midstream
A4 = Gorssel Downmidstream
A5 = Deventer Downstream
0_RFR = Project Olburgen
1_RFR = Project Havikerwaard
2_RFR = project Tichelbeekse
3_RFR = Project Welsummer
4_RFR = Obstakelsverwijderen
dan 0 of 1 of 2 is de timestep waarin het wordt geddaan
'''



def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "A.3_DikeIncrease 0": 5, "A.1_DikeIncrease 0": 5}
        )
    ),
        Policy(
        "policy x",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1}
        )
    ),
    
    # Policy(
    #     "policy 2",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 3",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 4",
    #     **dict(
    #         get_do_nothing_dict(),
            
    #     )
    # ),
]

In [28]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 20
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] performing 20 scenarios * 2 policies * 1 model(s) = 40 experiments
  0%|                                                   | 0/40 [00:00<?, ?it/s][MainProcess/INFO] performing experiments sequentially


100%|██████████████████████████████████████████| 40/40 [00:09<00:00,  4.21it/s]
[MainProcess/INFO] experiments finished


In [29]:
experiments, outcomes = results

In [30]:
# only works because we have scalar outcomes
outcomes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
outcomes=pd.DataFrame(outcomes)
outcomes

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,1.261845e+08,0.112792,6.302048e+07,0.056237,1.756107e+06,0.000881,0.000000e+00,0.000000,345900000.0,0.0
1,5.397251e+07,0.000000,5.541320e+06,0.005931,2.879840e+07,0.000000,9.658380e+06,0.005996,4.846158e+08,0.476760,345900000.0,0.0
2,5.397251e+07,0.000000,2.315577e+08,0.199443,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,345900000.0,0.0
3,5.397251e+07,0.000000,5.765934e+06,0.005989,2.879840e+07,0.000000,6.093805e+07,0.031370,0.000000e+00,0.000000,345900000.0,0.0
4,5.397251e+07,0.000000,2.176880e+07,0.020381,2.879840e+07,0.000000,1.317402e+07,0.007086,1.621455e+08,0.146502,345900000.0,0.0
5,5.397251e+07,0.000000,1.036557e+08,0.083441,2.879840e+07,0.000000,6.339506e+07,0.025815,5.390348e+08,0.420702,345900000.0,0.0
6,5.397251e+07,0.000000,6.164761e+06,0.005937,2.879840e+07,0.000000,1.070011e+07,0.005962,3.382875e+07,0.030819,345900000.0,0.0
7,5.397251e+07,0.000000,2.571949e+08,0.189564,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,345900000.0,0.0
8,5.397251e+07,0.000000,1.767352e+07,0.020468,2.879840e+07,0.000000,1.480056e+06,0.001014,3.581520e+08,0.391993,345900000.0,0.0
9,5.397251e+07,0.000000,0.000000e+00,0.000000,2.879840e+07,0.000000,9.001844e+07,0.051700,7.165079e+08,0.768002,345900000.0,0.0


In [31]:
print(np.mean(outcomes["A.2 Total Costs"]))
print(np.mean(outcomes["A.1_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.2_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.3_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.4_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.5_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.1_Expected Number of Deaths"].iloc[20:40].mean()))
print(np.mean(outcomes["A.2_Expected Number of Deaths"].iloc[20:40].mean()))
print(np.mean(outcomes["A.3_Expected Number of Deaths"].iloc[20:40].mean()))
print(np.mean(outcomes["A.4_Expected Number of Deaths"].iloc[20:40].mean()))
print(np.mean(outcomes["A.5_Expected Number of Deaths"].iloc[20:40].mean()))

121879511.01053569
0.0
0.08266350224385835
0.002811861201302967
0.017440619348163953
0.16467318555294133
0.8124267420938818
0.12303003377509607
0.7511758236425189
0.0030594744133828787
0.10399940080577419


In [32]:
experiments = pd.DataFrame(experiments)
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,EWS_DaysToThreat,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,111,289.780015,1.5,0.771663,46.371692,1.5,0.162438,105.205320,1.5,0.003039,82.845763,1.5,0.586384,190.210027,10.0,0.992937,3.5,3.5,1.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,32,policy 1,dikesnet
1,69,83.209870,1.5,0.866285,239.999544,1.5,0.734979,300.146470,10.0,0.855648,257.725537,1.5,0.798920,251.655636,10.0,0.178073,4.5,2.5,2.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,33,policy 1,dikesnet
2,56,271.365119,10.0,0.100197,98.064949,1.0,0.125817,347.996887,1.0,0.531975,68.149579,1.0,0.644107,220.786709,1.0,0.531455,2.5,4.5,1.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,34,policy 1,dikesnet
3,123,220.666941,10.0,0.357405,294.950470,1.5,0.846351,210.427495,1.0,0.479480,119.649735,10.0,0.376171,222.992603,1.5,0.718649,1.5,4.5,3.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,35,policy 1,dikesnet
4,117,37.793204,10.0,0.209136,210.852669,1.5,0.568267,278.362829,1.0,0.263885,143.199905,10.0,0.666770,314.845614,10.0,0.495717,2.5,1.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,36,policy 1,dikesnet
5,80,97.008952,10.0,0.594897,121.463900,10.0,0.208479,85.083696,1.5,0.139894,199.037552,10.0,0.249681,322.104646,1.0,0.122719,1.5,1.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,37,policy 1,dikesnet
6,34,76.837328,10.0,0.474433,158.213789,10.0,0.892366,30.680191,1.0,0.191511,244.919927,1.0,0.724314,280.226962,1.5,0.815746,4.5,1.5,2.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,38,policy 1,dikesnet
7,51,55.781312,1.0,0.706283,327.277700,1.0,0.098248,156.049127,10.0,0.748675,209.692358,10.0,0.861700,75.113679,10.0,0.300931,1.5,1.5,3.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,39,policy 1,dikesnet
8,91,224.677339,1.0,0.680318,126.406589,1.5,0.495635,54.438327,10.0,0.932940,287.506760,1.0,0.802673,89.838223,10.0,0.228779,3.5,3.5,3.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,40,policy 1,dikesnet
9,19,125.384602,1.0,0.253554,156.600720,1.0,0.934912,309.381068,1.5,0.631839,39.881628,10.0,0.142254,286.026124,1.0,0.030426,4.5,2.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,41,policy 1,dikesnet
